In [34]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from helper import nn
from helper import logistic_regression as lr

In [30]:
raw_X, raw_y = nn.load_data()
raw_X.shape

(5000, 400)

# prepare data

In [31]:
# add intercept=1 for x0
X = np.array([np.concatenate([np.ones(1), x]) for x in raw_X])
X.shape

(5000, 401)

In [32]:
# y have 10 categories here. 1..10, they represent digit 0 as category 10 because matlab index start at 1
# I'll ditit 0, index 0 again
y_matrix = []

for k in range(1, 11):
    y_matrix.append((raw_y == k).astype(int))

# last one is k==10, it's digit 0, bring it to the first position
y_matrix = [y_matrix[-1]] + y_matrix[:-1]
y = np.array(y_matrix)

y.shape

(10, 5000)

# train 1 model

In [35]:
t0 = lr.logistic_regression(X, y[0])

In [40]:
print(t0.shape)
y_pred = lr.predict(X, t0)
print('Accuracy={}'.format(np.mean(y[0] == y_pred)))

(401,)
Accuracy=0.9974


Is this real......

# train k model